In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# El data frame es de 3 columnas : PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen(url).read()

In [3]:
soup = BeautifulSoup(source, 'html.parser')

In [4]:
## Datos de la cabecera de la tabla ####

p = soup.find_all('th')
paragraphs = []
for x in p:
    paragraphs.append(str(x))

postcode = paragraphs[0][4:12]
borough  = paragraphs[1][4:11]
vecinda  = paragraphs[2][4:17]


In [5]:
#### Datos de la tabla #####

table = soup.find('table')
table_rows = table.find_all('tr')
l = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

data = pd.DataFrame(l, columns=[postcode, borough, vecinda])

data= data[data.Borough != 'Not assigned']
data.dropna(inplace=True)

data = data[[postcode,borough,vecinda]]

In [6]:
# iterate over the dataframe row by row
for index_label, row_series in data.iterrows():
   # For each row update the 'Bonus' value to it's double
   data.at[index_label , 'Neighbourhood'] = row_series['Neighbourhood'].replace("\n", "")

In [7]:
data.loc[data['Neighbourhood']=='Not assigned', ['Neighbourhood']] = 'Queen\'s Park'

In [8]:
data.Neighbourhood = data.Neighbourhood.replace({"Harbourfront": "Regent Park,Harbourfront"})
data = data[data.Neighbourhood != 'Regent Park']

In [9]:
print(data)

    Postcode           Borough                                      Neighbourhood
3        M3A        North York                                          Parkwoods
4        M4A        North York                                   Victoria Village
5        M5A  Downtown Toronto                           Regent Park,Harbourfront
7        M6A        North York                                   Lawrence Heights
8        M6A        North York                                     Lawrence Manor
9        M7A      Queen's Park                                       Queen's Park
11       M9A         Etobicoke                                   Islington Avenue
12       M1B       Scarborough                                              Rouge
13       M1B       Scarborough                                            Malvern
15       M3B        North York                                    Don Mills North
16       M4B         East York                                   Woodbine Gardens
17       M4B    